## Issues New
This is the reference implementation for [Issues New](https://github.com/chaoss/wg-evolution/blob/master/metrics/Issues_New.md),
a metric specified by the
[Evolution Working Group](https://github.com/chaoss/wg-evolution) of the
[CHAOSS project](https://chaoss.community).
This implementation is specific to Git repositories.

Have a look at [README.md](../README.md) to find out how to run this notebook (and others in this directory) as well as to get a better understanding of the purpose of the implementations.

The implementation is described in two parts (see below):

* Class for computing Issues New
* An explanatory analysis of the class' functionality

Some more auxiliary information in this notebook:

* Examples of the use of the implementation

As discussed in the [README](../README.md) file, the scripts required to analyze the data fetched by Perceval are located in the `code_df` package. Due to python's import system, to import modules from a package which is not in the current directory, we have to either add the package to `PYTHONPATH` or simply append a `../..` to `sys.path`, so that `code_df` can be successfully imported. 

In [29]:
from datetime import datetime
import matplotlib.pyplot as plt

import sys
sys.path.append('../..')

from implementations.scripts.issue_github import IssueGithub
from implementations.scripts.utils import read_json_file

In [30]:
class IssueNewGithub(IssueGithub):
    """
    Issues New
    """

    def compute(self):
        """
        Compute the number of new issues in the Perceval data.

        :returns: the number of issues created
        """
        # new_issues = [item['hash'] for item in self.items]
        # return len(new_issues)
        return len(self.items)

    def __str__(self):
        return 'Issues New'

## Performing the Analysis
Using the above class, we can perform several kinds of analysis on the JSON data file, fetched by Perceval.

For starters, we can perform a simple count of the number of new issues created in the data. For this analysis, we can vary the value passed to the `date_range` parameter.

### Counting the total number of commits 
We first read the JSON file containing Perceval data using the `read_json_file` utility function. 

In [31]:
items = read_json_file('../issues_events.json')

Let's use the `compute` method to count the total number of issues created. First, we will do it without passing any since and until dates. 
Next, we can pass in the start and end dates as a tuple. The format would be `%Y-%m-%d`.

A third kind of analysis we can perform is passing only one of the dates to `date_range` --- either `since` or `until`.

In [32]:
issues = IssueNewGithub(items)
print('The total number of issues '
        'in the file is {}.'.format(issues.compute()))

date_since = datetime.strptime("2018-01-01", "%Y-%m-%d")
date_until = datetime.strptime("2019-07-01", "%Y-%m-%d")

issues_dated = IssueNewGithub(items, date_range=(date_since, date_until))

print("The total number of issues created between 2018-01-01 and 2019-07-01 is {}.".format(issues_dated.compute()))

issues_after = IssueNewGithub(items, date_range=(date_since, None))

print("The total number of issues created after 2018-01-01 is {}.".format(issues_after.compute()))

The total number of issues in the file is 241.
The total number of issues created between 2018-01-01 and 2019-07-01 is 193.
The total number of issues created after 2018-01-01 is 241.


## Reopen as New
When this parameter is `True`, every time an issue is reopened, it is treated as a new issue. From the creation of the issue to the first closing event is considered as a separate issue. After that, every reopening creates a new issue. Finally, the original issue which we considered is removed, since it is now redundant.

To use this, simply pass `True` for this parameter in the class' `__init__` method.

In [33]:
issues = IssueNewGithub(items, reopen_as_new=True)
print("The total number of issues, condering reopened issues as new is:"
        "in the file is {}.".format(issues.compute()))

date_since = datetime.strptime("2018-01-01", "%Y-%m-%d")
date_until = datetime.strptime("2019-07-01", "%Y-%m-%d")

issues_dated = IssueNewGithub(items,
                        date_range=(date_since, date_until),
                        reopen_as_new=True)

print("The total number issues created between 2018-01-01 and 2019-07-01, "
        "considering reopened issues as new is {}.".format(issues_dated.compute()))

The total number of issues, condering reopened issues as new is:in the file is 248.
The total number issues created between 2018-01-01 and 2019-07-01, considering reopened issues as new is 200.
